In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys, gzip
import hpiptools as ht
import mybiotools as mbt

# 2018-06-21 Preparing "collect_integrations"

In this notebook I want to prepare the final part of the pipeline, which consists in two separate steps:

1. Collect the mapping and counting information from each library. This is an easy step, in which I only need to parse relatively easy files.
2. Assign a promoter to each barcode. This is the difficult step. To do this, I need to harness all the information that I have, which currently is:
    - what does the promoter-barcode dictionary says
    - what iPCR, cDNA, and gDNA index say

The latter step is the most complicated, because potentially the first assignment of the read to a particular library, based on the index, were not done correctly. Therefore, I want to store all the information that I possibly have into a data structure that will allow to have an idea of how to assign the promoter in the best possible way.

In [ ]:
# HPIP root directory
hpip_root = '%s/work/CRG/projects/hpip'%(os.getenv('HOME'))

# prepare the names of the libraries
libs = ['lib%d'%(i) for i in range(2, 13)]
libs.append('undetermined')

In [ ]:
class Replicate :
    def __init__(self, rep_name) :
        self.rep_name = rep_name

In [ ]:
rep_names = ['rep1', 'rep2']
reps = []
for rep_name in rep_names :
    reps.append(Replicate(rep_name))

In [ ]:
for rep in reps :
    # now we parse ALL the starcoded cDNA and gDNA output files
    cDNA_canonicals = {}
    cDNA_counts = {}
    gDNA_canonicals = {}
    gDNA_counts = {}
    for lib in libs :
        libdir = '%s/data/triplibs/%s/%s'%(hpip_root, rep.rep_name, lib)
        cDNA_starcode_fname = '%s/cDNA-starcode.txt'%(libdir)
        gDNA_starcode_fname = '%s/gDNA-starcode.txt'%(libdir)
        if not os.path.exists(cDNA_starcode_fname) or\
           not os.path.exists(gDNA_starcode_fname) :
                continue
        ht.log_message(rep.rep_name, 'Parsing %s starcode'%(lib))
        cDNA_canonicals[lib], cDNA_counts[lib] =\
                   ht.parse_starcode(cDNA_starcode_fname)
        gDNA_canonicals[lib], gDNA_counts[lib] =\
                   ht.parse_starcode(gDNA_starcode_fname)
    
    # assign the structure to the replicate
    rep.cDNA_canonicals = cDNA_canonicals
    rep.cDNA_counts = cDNA_counts
    rep.gDNA_canonicals = gDNA_canonicals
    rep.gDNA_counts = gDNA_counts    

In [ ]:
for rep in reps :
    iPCR_canonicals = {}
    iPCR_counts = {}
    mapped = {}
    for lib in libs :
        # data directory for the library under study
        lib_dir = '%s/data/triplibs/%s/%s'%(hpip_root, rep.rep_name, lib)

        # build file names
        iPCR_starcode_fname = '%s/iPCR-starcode.txt'%(lib_dir)
        iPCR_sam_fname = '%s/iPCR.sam'%(lib_dir)

        # work on the starcoded files
        ht.log_message(rep.rep_name, 'Processing %s iPCR starcode'%(lib))
        iPCR_canonicals[lib], iPCR_counts[lib] =\
                  ht.parse_starcode(iPCR_starcode_fname)
            
        # now we can open the mapped file
        ht.log_message(rep.rep_name, 'Processing %s mapped file'%(lib))
        mapped[lib], nmulti, nmapped, nunmapped =\
             ht.parse_mapped(iPCR_sam_fname)
    rep.iPCR_canonicals = iPCR_canonicals
    rep.iPCR_counts = iPCR_counts
    rep.mapped = mapped

In [ ]:
nlibs = len(libs)
for rep in reps :
    
    rep.cDNA_transitions = np.zeros((nlibs, nlibs), dtype=np.int32)
    rep.gDNA_transitions = np.zeros((nlibs, nlibs), dtype=np.int32)
    
    # go through all the libraries
    for i, lib_origin in enumerate(libs) :
        
        ht.log_message(rep.rep_name, 'Processing %s'%(lib_origin))
        
        # go through all the integrations in that library
        for bcd, integration in rep.mapped[lib_origin].iteritems() :
            try :
                canonical = rep.iPCR_canonicals[lib_origin][bcd]
            except KeyError :
                # this occurs if a barcode was removed from the output
                # sequences of starcode because of ambiguous cluster 
                # assignment.
                continue

            # now we try to find a match between the canonical and the cDNA
            # and gDNA canonicals in ALL other libraries
            for j, lib in enumerate(libs) :
                if rep.cDNA_canonicals[lib].has_key(canonical) :
                    rep.cDNA_transitions[i,j] += 1
                if rep.gDNA_canonicals[lib].has_key(canonical) :
                    rep.gDNA_transitions[i,j] += 1

In [ ]:
# figure preparation
labels = [s[:5] for s in libs]
for rep in reps :
    fig, axes = plt.subplots(2, 1, figsize=(8, 16))
    x = range(len(libs))

    # cDNA
    ax = axes[0]
    ax.matshow(mbt.row_normalize_matrix(rep.cDNA_transitions.astype('float')))
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=90)
    ax.set_yticks(x)
    ax.set_yticklabels(labels)
    ax.set_ylabel('cDNA')
    ax.set_title(rep.rep_name, y=1.1, fontsize=32)

    # gDNA
    ax = axes[1]
    ax.matshow(mbt.row_normalize_matrix(rep.gDNA_transitions.astype('float')))
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=90)
    ax.set_yticks(x)
    ax.set_yticklabels(labels)
    ax.set_ylabel('gDNA')
    
    plt.show()

## Assigning the promoter to each integration

Now that all of this is done, it's more clear what's going on. There's a part which is the easy part: replicate 1. In replicate 2, there's more work to do to assign the library index correctly.

In [ ]:
# init promoter-barcode dictionary
pbd = ht.PBD(hpip_root)

In [ ]:
prom_libs = range(1,13)
prom_lib_idx = {str(prom_lib) : prom_lib-1 for prom_lib in prom_libs}
prom_class_idx = {'A':0,'B':1,'C':2,'D':3,
              'E':4,'F':5,'G':6,'H':7}

In [ ]:
promoters = np.zeros((8,12))
n = 0
mapped = reps[0].mapped['lib3']
for bcd in mapped.iterkeys() :
    n += 1
    candidates = pbd.findbcd(bcd)
    if candidates is None :
        continue
    candidates = ht.parse_bcd(candidates)
    for candidate in candidates.iterkeys() :
        prom_class, prom_lib = ht.prom_id(candidate)
        promoters[prom_class_idx[prom_class], prom_lib_idx[prom_lib]] += 1

In [ ]:
cax = plt.matshow(np.log10(promoters+1))
plt.xticks(prom_lib_idx.values(), prom_lib_idx.keys())
plt.yticks(prom_class_idx.values(), prom_class_idx.keys())
plt.colorbar(cax)
plt.show()

In [ ]:
plt.bar(range(12), np.log10(promoters.sum(axis=0)))
plt.xticks(prom_lib_idx.values(), prom_lib_idx.keys())
plt.xlabel('Library ID')
plt.ylabel('Log_10 Counts')
plt.show()

This works well in some cases, much less so in other cases. What we now need to do is to find a way of assigning a barcode to a promoter, based on all the information we collected so far. A first preparatory step for this process is to merge all the integrations into one, if they have the same integration site. Let's have a look at the distribution of counts.

In [ ]:
rep = reps[1]
lib = 'lib7'
ninsertions = []
for bcd, insertion in rep.mapped[lib].iteritems() :
    if rep.cDNA_counts[lib].has_key(bcd) and\
       rep.gDNA_counts[lib].has_key(bcd) :
        ninsertions.append(insertion[1])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,4))
ax.hist(ninsertions,bins=100)
ax.set_yscale('log')
ax.set_xlabel('Number of integrations')
ax.set_ylabel('Counts')
plt.show()

This is fine, the number of integrations has a kind of expected decay. Now I prepare the functions that will allow me to sanitize the barcode list, before finally writing the output to the final files.

In [ ]:
rep = reps[1]
lib_id = 7
lib = 'lib%d'%(lib_id)
n = 0
with open('test.txt', 'w') as f :
    for bcd, insertion in rep.mapped[lib].iteritems() :
        
        # define flags that will allow us to make some statistics
        # about how many integration get in and out of the final
        # insertion table
        in_cDNA = rep.cDNA_counts[lib].has_key(bcd)
        in_gDNA = rep.gDNA_counts[lib].has_key(bcd)
        in_both = in_cDNA and in_gDNA
        
        # interrogate the pbd to know whether the barcode that we
        # are looking at was actually known in the promoter-barcode
        # association table
        pr = pbd.findbcd(bcd)
        in_pbd = pr is not None
        
        # proceed with the calculations only if the three conditions
        # are met: the mapped insertion is in the cDNA, in the gDNA, 
        # and in the pbd.
        if in_cDNA and in_gDNA and in_pbd :
            n += 1
            
            # get the list of promoters associated to the
            # barcode from the pbd
            proms = ht.parse_bcd(pr)
            in_pbd_lib = proms.has_key(lib_id)
            
            # if we find that the barcode was in the pbd, but none of
            # the candidate promoters are in the library under consideration,
            # then we need to throw this integration away because we cannot
            # know which promoter it is associated to
            if not in_pbd_lib :
                continue

            # if we are here, everything is fine and we prepare the string
            # that we output to our final file. First, we prepare a string
            # that contains the information about the candidate promoters
            prom_string = ''
            for prom_class, p in proms[lib_id] :
                prom_string += '\t%s%d:%s'%(prom_class, lib_id, p)
            
            # we then prepare the line that contains the information about
            # the barcode, the mapping, the reads, and the promoter
            chrom, pos, strand = insertion[0]
            iPCR_counts = insertion[1]
            line = '%s\t'%(bcd)
            line += '%s\t%d\t%s'%(chrom, pos, strand)
            line += '\t%d\t%d\t%d'%(iPCR_counts,
                                   rep.cDNA_counts[lib][bcd],
                                   rep.gDNA_counts[lib][bcd])
            line += prom_string
            line += '\n'
            f.write(line)
            # print line
        # if n>=10 : break

And with this, we're ready to write the collect_integrations script.